In [1]:
# https://www.kaggle.com/c/feedback-prize-2021/data train.zip по этой ссылке

## Код подготовки данных для KenLM

В качестве данных используются [Feedback Prize](https://www.kaggle.com/c/feedback-prize-2021/data)
и данные с Решу ОГЭ/ЕГЭ

In [9]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [10]:
%%capture
!unzip /content/train.zip

In [11]:
%%capture
!unzip -u '/content/drive/MyDrive/nto_final_data.zip'

In [12]:
import pandas as pd
from sklearn.model_selection import train_test_split
from tqdm.notebook import tqdm
import re
import os
import pickle

In [13]:
alphabet = """@ !"%'()+,-./0123456789:;=?AEFIMNOSTW[]abcdefghiklmnopqrstuvwxyАБВГДЕЖЗИКЛМНОПРСТУХЦЧШЭЮЯабвгдежзийклмнопрстуфхцчшщъыьэюяё№"""

In [15]:
# мы спарсили школьные задания для сочинений в этом блокноте parser_sdamgia.ipynb
with open('drive/MyDrive/sdamgia_339.pkl', 'rb') as f:
    ru_data1 = list(pickle.load(f))

ru_data = []
for i in tqdm(ru_data1):
    l = list(list(i)[1])
    for j in l:
        j2 = re.sub(r"\(\d+\)", "", j)
        words = j2.split()
        ru_data += words

en_data = []
for i in tqdm(os.listdir('/content/')):
    if '.txt' in i:
        en_text = open(i).read()
        en_words  = en_text.split()
        en_data += en_words 

  0%|          | 0/339 [00:00<?, ?it/s]

  0%|          | 0/15600 [00:00<?, ?it/s]

In [16]:
labels = pd.read_csv('/content/data/train_recognition/labels.csv')
labels_train, labels_val = train_test_split(labels, test_size=0.04, random_state=655)

In [17]:
def prepare_word(word):
    new_word = []
    for c in word:
        if c in alphabet:
            if c == ' ':
                new_word.append('|')
            else:
                new_word.append(c)
        else:
            new_word.append('@')
    return " ".join(new_word)

In [18]:
def create_dataset(words, name):
    with open(name, 'a') as out:
        for word in words:
            out.write(prepare_word(word) + '\n')

In [19]:
create_dataset(list(labels_train.text.values) + ru_data + en_data, '/content/drive/MyDrive/nto_dataset11.txt')

In [20]:
# тренировали мы kenlm в линуксе в своей системе с помощью
# bin/lmplz --discount_fallback -o 6  < nto_dataset11.txt > nto_kenlm_model10.arpa